In [1]:
import os
import dotenv

dotenv.load_dotenv()

True

In [3]:
from llama_index.core import SimpleDirectoryReader
from llama_index.readers.file import PyMuPDFReader
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import TokenTextSplitter

from tempfile import TemporaryDirectory
import os
import base64

from llama_index.multi_modal_llms.anthropic import AnthropicMultiModal
from anthropic import AnthropicBedrock
import pymupdf

print("AWS_PROFILE:", os.getenv("AWS_PROFILE"))
client = AnthropicBedrock()
# pages = []

with open("./datasets/cds/mississippi-state.md", "w") as fp:
    doc = pymupdf.open('./datasets/cds/pdf/Mississippi State University CDS_2023-2024.pdf')

    for page in doc:  # iterate through the pages
        pix = page.get_pixmap()  # render page to an image
        # pix.save(os.path.join(tempdir, "page-%i.png" % page.number))  # store image as a PNG
        encoded_string = base64.b64encode(pix.tobytes())
        # print(encoded_string)

        prompt = """
        Convert this image of a page from a PDF into markdown.
        Convert the tables to valid markdown tables.
        If there is an image or figure, replace it with an alternative text description.
        Return the formatted markdown in a block starting with "<MARKDOWN>" and ending with "</MARKDOWN>"
        """
    
        message = client.messages.create(
            model="anthropic.claude-3-5-sonnet-20240620-v1:0",
            # model="anthropic.claude-3-haiku-20240307-v1:0",
            max_tokens=2048,
            messages=[
                {"role": "user", "content": [
                  {
                    "type": "image",
                    "source": {
                      "type": "base64",
                      "media_type": "image/png",
                      "data": encoded_string.decode('utf-8'),
                    }
                  },
                  {"type": "text", "text": prompt}
                ]}
            ]
        )

        content = message.content

        if content != [] and content[0].type == 'text':
            text = content[0].text
            print("before:", text)

            sub1 = "<MARKDOWN>"
            sub2 = "</MARKDOWN>"

            idx1 = 0
            idx2 = len(text)
            # print(text)
            try:
                idx1 = text.index(sub1)
            except ValueError:
                pass

            try:
                idx2 = text.index(sub2)
            except ValueError:
                pass

            # length of substring 1 is added to
            # get string from next character
            res = text[idx1 + len(sub1) + 1: idx2]

            print(res)
            fp.write(res)
    

# image_documents = SimpleDirectoryReader(input_files=[img_file]).load_data()
# response = anthropic_mm_llm.complete(
#     prompt="Describe the images as an alternative text",
#     image_documents=image_documents,
# )

# reader = PyMuPDFReader()

# # load document
# # documents = reader.load_data()
# documents = reader.load_data(file_path='./datasets/cds/pdf/Mississippi State University CDS_2023-2024.pdf')
# # documents = []

# # for doc in reader.iter_data():
# #     for page in doc:
# #         print("page:", page)

# #     documents.extend(doc)

# # load nodes
# pipeline = IngestionPipeline(transformations=[TokenTextSplitter()])

# nodes = pipeline.run(documents=documents)

AWS_PROFILE: collega-prod
before: Here is the markdown conversion of the image:

<MARKDOWN>
# Mississippi State University
## Office of Institutional Research and Effectiveness

### A. General Information

#### A0. Respondent Information

| Field | Value |
|-------|-------|
| Office: | Office of Institutional Research and Effectiveness |
| Address: | P.O. Drawer EY |
| City: | Mississippi State |
| State: | Mississippi |
| Zip: | 39762 |
| Country: | United States |
| Phone Number: | 662-325-3920 |
| Email Address: | oire@ir.msstate.edu |

Are your responses to the CDS posted for reference on your institution's website?
Yes

If yes, please provide a direct link to the posted CDS responses:
https://ir.msstate.edu/cdsets.php

#### A1. Address Information
Please enter general institution information below:

| Field | Value |
|-------|-------|
| Name of College or University | Mississippi State University |
| Street Address: | 75 B.S. Hood Road |
| City: | Mississippi State |
| State: | Mi

KeyboardInterrupt: 

In [46]:
documents

[ImageDocument(id_='cb8fded7-9af0-45fb-9ce5-683463d80538', embedding=None, metadata={'file_path': '/var/folders/fb/y8c4qgzx1qx0nnnhwcvm9h4c0000gp/T/tmp3zt_4yhs/page-0.png', 'file_name': 'page-0.png', 'file_type': 'image/png', 'file_size': 101953, 'creation_date': '2024-06-25', 'last_modified_date': '2024-06-25'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n', image=None, image_path='/var/folders/fb/y8c4qgzx1qx0nnnhwcvm9h4c0000gp/T/tmp3zt_4yhs/page-0.png', image_url=None, image_mimetype=None, text_embedding=None),
 ImageDocument(id_='337a066c-9eff-4a7a-95c3-2bd9db3e6abc', embedding=None, metadata={'file_path

In [39]:
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.bedrock import BedrockEmbedding
from llama_index.embeddings.openai import OpenAIEmbedding

# BedrockEmbedding.list_supported_models()

# embed_model = BedrockEmbedding(
#     model="cohere.embed-english-v3",
#     profile_name="default",
# )
embed_model = OpenAIEmbedding(
    model="text-embedding-3-small",
    api_key=os.environ["OPENAI_API_KEY"]
)

index = VectorStoreIndex(nodes, embed_model=embed_model)

In [41]:
# from llama_index.llms.bedrock_converse import BedrockConverse
from llama_index.multi_modal_llms.openai import OpenAIMultiModal

llm = BedrockConverse(
    model="anthropic.claude-3-haiku-20240307-v1:0",
    profile_name="default",
)
# llm = OpenAIMultiModal(
#     model="gpt-4o",
#     api_key=os.environ["OPENAI_API_KEY"]
# )

chat_engine = index.as_chat_engine(llm=llm, verbose=True)
chat_engine.chat("What does this school offer masters degrees?")

system header: You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.

## Tools

You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.

You have access to the following tools:
{tool_desc}


## Output Format

Please answer in the same language as the question and use the following format:

```
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a Thought.

Please use a valid JSON format for the Action Input. Do NOT do this {{'input': 'hello world'

AgentChatResponse(response="Understood. The context provided does not contain any information about whether Mississippi State University offers master's degrees. The information is focused on the undergraduate academic offerings and bachelor's degrees, but does not mention anything about graduate programs or master's degrees. Without additional information about the university's graduate-level offerings, I cannot definitively answer whether they offer master's degrees. Please let me know if you have any other questions!", sources=[ToolOutput(content='Thought: The current language of the user is English. I need to use a tool to help me answer the question.\nAction: query_engine_tool\nAction Input: {"input": "What does this school offer masters degrees?"}\n\nObservation: Based on the information provided in the context, the document does not mention anything about the school offering master\'s degrees. The context focuses on the undergraduate academic offerings and policies, as well as t

In [42]:
for doc in documents:
    for page in doc:
        if page[0] == 'text':
            print(llm.complete(f"convert this page to markdown: \n\n{page[1]}"))

Thought: The current language of the user is: English. I need to use a tool to help me convert the provided information to Markdown format.

Action: query_engine_tool
Action Input: {"input": "# Mississippi State University\n\n## Office of Institutional Research and Effectiveness\n\n### A0. Respondent Information\n\nOffice of Institutional Research and Effectiveness\nP.O. Drawer EY\nMississippi State\nMississippi\n39762\nUnited States\n662-325-3920\noir@ir.msstate.edu\n\nAre your responses to the CDS posted for reference on your institution's website?\nYes\n\nIf yes, please provide a direct link to the posted CDS responses:\nhttps://ir.msstate.edu/cdsets.php\n\n### A1. Address Information\n\nPlease enter general institution information below:\n\n**Name of College or University**\nMississippi State University\n\n**Street Address:**\n75 B.S. Hood Road\n\n**City:**\nMississippi State\n\n**State:**\nMississippi\n\n**Zip:**\n39762\n\n**Country:**\nUnited States\n\n**Main Institution Phone Nu

KeyboardInterrupt: 